# Live Sign Language Translator
Our goal is to create a program that will recognise sign language caught by video and translate it into it's English counterpart.
#### Current Limitations:
- Only recognises ASL Letters and not any words.
- Dataset does not include J and Z since they require motion. Current workaround is to put a placeholder and try to guess the word while generating words

In [1]:
import numpy as np
import pandas as pd # data processing
import time
import matplotlib.pyplot as plt
import torch

import tensorflow as tf

from asl_dataset import SignLanguageDataset
from asl_net import Net

## Initialize net from predefined Neural Net

In [2]:
net = Net()
labels = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 
          'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y']

net_path = "sign_lang_net.pth"

net.load_state_dict(torch.load(net_path))

net.eval()

Net(
  (layer1): Sequential(
    (0): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (3): ReLU()
  )
  (layer2): Sequential(
    (0): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (3): ReLU()
  )
  (fc1): Linear(in_features=400, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=64, bias=True)
  (leak): LeakyReLU(negative_slope=0.01)
  (drop): Dropout(p=0.3, inplace=False)
  (fc3): Linear(in_features=64, out_features=24, bias=True)
)

## Initalize MediaPipe Hands

In [3]:
import numpy
import cv2
import mediapipe as mp
cap = cv2.VideoCapture(0)
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_draw = mp.solutions.drawing_utils

I0000 00:00:1705455853.057898       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 83.1), renderer: Apple M2


## Define function to read from camera and return letter

In [11]:
def get_letter():
    success, image = cap.read()
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    results = hands.process(image_rgb)
    
    if results.multi_hand_landmarks != None:
        landmarks = results.multi_hand_landmarks[0].landmark

        #get_bounds
        x_min, y_min, x_max, y_max = crop_dimensions(image, landmarks)
        
        # Crop the image to the bounding box
        crop = image[y_min:y_max, x_min:x_max]
        crop = cv2.resize(crop,(128, 128), interpolation=cv2.INTER_CUBIC)

        img_tensor = torch.tensor(crop).unsqueeze(0).float()
        print(img_tensor)
        letter = net(img_tensor.unsqueeze(0)).squeeze(0).softmax(0)
        print(letter)
    
    return ''
        

#returns x_min, y_min, len
def crop_dimensions(image, lms):
    x_coords = np.array([lm.x for lm in lms])
    y_coords = np.array([lm.y for lm in lms])
    
    # Calculate bounding box
    margin = 50
    x_min = max(0, int(np.min(x_coords) * image.shape[1]) - margin)
    y_min = max(0, int(np.min(y_coords) * image.shape[0]) - margin)
    x_max = min(image.shape[1], int(np.max(x_coords) * image.shape[1]) + margin)
    y_max = min(image.shape[0], int(np.max(y_coords) * image.shape[0]) + margin)
    
    # Ensure a square bounding box
    length = max(x_max - x_min, y_max - y_min)
    x_max = min(image.shape[1], x_min + length)
    y_max = min(image.shape[0], y_min + length)
    
    return x_min, y_min, x_max, y_max


In [12]:
get_letter()

tensor([[[160., 161., 161.,  ..., 255., 255., 255.],
         [153., 153., 152.,  ..., 255., 255., 255.],
         [148., 150., 149.,  ..., 255., 255., 255.],
         ...,
         [ 81.,  83.,  79.,  ..., 108., 107., 113.],
         [ 80.,  78.,  77.,  ..., 105.,  94.,  89.],
         [ 76.,  73.,  81.,  ...,  98.,  95.,  98.]]])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x14400 and 400x100)